In [1]:
import numpy as np
import cv2
from cv2 import dnn
cv2.__version__

'4.7.0'

In [2]:
#load yolo
net = cv2.dnn.readNet("yolov3.weights","yolov3.cfg") #read file

#crate list class read form coco.names file
classes = []

with open("coco.names","r") as f:
    classes = [line.strip() for line in f.readlines()] # data list

layer_names = net.getLayerNames()
len(classes)

80

In [3]:
classes[:10]
classes

['person',
 'bicycle',
 'car',
 'motorbike',
 'aeroplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'sofa',
 'pottedplant',
 'bed',
 'diningtable',
 'toilet',
 'tvmonitor',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']

In [4]:
# สร้างค่าสีแบบสุ่มเพื่อเขียนกรอบสี่เหลี่ยมรอบวัตถุ (bounding box)
colors = np.random.uniform(0,255,size=(len(classes),3))

In [5]:
img = cv2.imread("dog.jpg")
img = cv2.resize(img, None, fx=0.8, fy=0.8) # scale image ย่อสเกลภาพ
heigth,width,channels =  img.shape
img.shape

(600, 883, 3)

In [6]:
img_size = (320,320)
blob = cv2.dnn.blobFromImage(img,1.0/255,img_size,(0,0,0),True,crop=False)
net.setInput(blob)

In [7]:
print(net.getUnconnectedOutLayers())

[200 227 254]


In [8]:
unconnect = net.getUnconnectedOutLayers()

In [9]:
output_layers = [layer_names[i-1] for i in unconnect]
outputs = net.forward(output_layers)
print(outputs[0])

[[0.04755429 0.05224197 0.36834425 ... 0.         0.         0.        ]
 [0.05591626 0.04215427 0.32421374 ... 0.         0.         0.        ]
 [0.06188939 0.04571395 1.0631661  ... 0.         0.         0.        ]
 ...
 [0.94738483 0.9351803  0.5285537  ... 0.         0.         0.        ]
 [0.9493453  0.95500576 0.42928228 ... 0.         0.         0.        ]
 [0.95809096 0.9554365  1.0670701  ... 0.         0.         0.        ]]


In [10]:
class_ids = []
confidences = []
boxes = []
for out in outputs:
    for detection in out:
        scores = detection[5:] #ค่า score ที่ทำนายของวัตถุ
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5 :
            center_x = int(detection[0]*width)
            center_y = int(detection[1]*heigth)
            w = int(detection[2]*width)
            h = int(detection[3]*heigth)

            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x,y,w,h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
indices = cv2.dnn.NMSBoxes(boxes,confidences,0.5,0.4)

In [11]:
font = cv2.FONT_HERSHEY_SIMPLEX
text_color = (255,255,255)

for i in range(len(boxes)):
    if i in indices :
        x,y,w,h = boxes[i]
        label = str(classes[class_ids[i]])
        conf = confidences[i]
        label = '{} {:1f}%'.format(label,conf*100)
        color = colors[i]
        
        t_size = cv2.getTextSize(label,font,1,1)[0]
        cv2.rectangle(img,(w,y),(x+w , y+h),color,2)
        
        c2 = x+int(t_size[0]*0.8),y-int(t_size[1]*1.2)
        cv2.rectangle(img,(x,y-2),c2,color,-1)
        cv2.putText(img,label,(x,y-2),font,0.7,text_color,3)

In [12]:
cv2.imshow("Mywindow",img)
cv2.waitKey(0)
cv2.destroyAllWindows()